In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mlflow
import polars as pl
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from torch import optim

from nn_trainer.torch.evaluator import MeanSquaredErrorEvaluator
from nn_trainer.torch.loader import MiniBatchLoader
from nn_trainer.torch.loss_fn import mean_squared_error
from nn_trainer.torch.model import RegressionMLP
from nn_trainer.torch.trainer import Trainer

In [3]:
housing = fetch_california_housing()
train_dataset_df, valid_dataset_df = train_test_split(
    pl.from_numpy(data=housing.data, schema=housing.feature_names).with_columns(  # type: ignore
        pl.Series(housing.target).alias(housing.target_names[0])  # type: ignore
    ),
    test_size=0.2,
    random_state=0,
)
train_X_df, train_y_df, valid_X_df, valid_y_df = (
    train_dataset_df.drop(housing.target_names[0]).cast(pl.Float32),  # type: ignore
    train_dataset_df.select(housing.target_names[0]).cast(pl.Float32),  # type: ignore
    valid_dataset_df.drop(housing.target_names[0]).cast(pl.Float32),  # type: ignore
    valid_dataset_df.select(housing.target_names[0]).cast(pl.Float32),  # type: ignore
)

In [4]:
model = RegressionMLP(train_dataset_df.shape[1] - 1, [10, 10], 1)
train_loader = MiniBatchLoader(X_df=train_X_df, y_df=train_y_df, batch_size=512, seed=0)
valid_evaluator = MeanSquaredErrorEvaluator(X_df=valid_X_df, y_df=valid_y_df)

### Training

In [5]:
%%time

mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("REGRESSION")

with mlflow.start_run() as run:
    mlflow.log_param("my", "param")

    trainer = Trainer(
        model=model,
        optimizer=optim.Adam(model.parameters(), lr=0.001),
        train_loader=train_loader,
        loss_fn=mean_squared_error,
        valid_evaluator=valid_evaluator,
        early_stopping_patience=10,
        epoch_num=512,
        active_run=run,
    )
    trainer = trainer.fit()

[VALID 000]: loss=129.85862731933594, metrics={'MSE': 129.85862731933594}


[TRAIN 001]: 100%|██████████| 32/32 [00:00<00:00, 530.54it/s, batch_loss=4.86]


[VALID 001]: loss=5.124122619628906, metrics={'MSE': 5.124122619628906}


[TRAIN 002]: 100%|██████████| 32/32 [00:00<00:00, 546.81it/s, batch_loss=2.1]


[VALID 002]: loss=2.525139808654785, metrics={'MSE': 2.525139808654785}


...

[VALID 161]: loss=0.5619193315505981, metrics={'MSE': 0.5619193315505981}


[TRAIN 162]: 100%|██████████| 32/32 [00:00<00:00, 565.33it/s, batch_loss=0.593]


[VALID 162]: loss=0.572329044342041, metrics={'MSE': 0.572329044342041}


[TRAIN 163]: 100%|██████████| 32/32 [00:00<00:00, 599.76it/s, batch_loss=0.573]


[VALID 163]: loss=0.5595735311508179, metrics={'MSE': 0.5595735311508179}
🏃 View run delicate-shark-591 at: http://localhost:8080/#/experiments/663538387065204330/runs/a22eab205e624575bda6f31b4ad71e79
🧪 View experiment at: http://localhost:8080/#/experiments/663538387065204330
CPU times: user 27.3 s, sys: 2.24 s, total: 29.5 s
Wall time: 13.8 s


### Inference

In [ ]:
valid_loader = MiniBatchLoader(X_df=train_X_df, y_df=train_y_df, batch_size=512, seed=0)
valid_loader.setup_epoch()
for Xs, y in valid_loader:
    break
trainer.best_model(*Xs)[:10]

tensor([[2.0524],
        [2.1204],
        [1.2183],
        [2.6473],
        [2.3307],
        [2.4153],
        [1.8353],
        [1.3988],
        [3.3848],
        [1.8919]], grad_fn=<SliceBackward0>)